In [1]:

import duckdb 
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb

In [4]:
path_data="../datasets/"
file_data_bruto = "competencia_01_crudo.csv"
path_data_bruto=path_data + file_data_bruto

In [5]:
path_data_bruto

'../datasets/competencia_01_crudo.csv'

In [7]:
%%sql
create or replace table df as
select *
from read_csv_auto('../datasets/competencia_01_crudo.csv')

,Count
0,978439


## 1er Intento

In [ ]:
%%sql
SELECT *,
CASE WHEN t1.foto_mes < 202105 AND t1.foto_mes+2 NOT IN 
    (SELECT t2.foto_mes
    FROM df t2
    WHERE t2.numero_de_cliente = t1.numero_de_cliente) THEN "BAJA+2"
WHEN t1.foto_mes < 202106 AND t1.foto_mes+1 NOT IN 
    (SELECT t2.foto_mes
    FROM df t2
    WHERE t2.numero_de_cliente = t1.numero_de_cliente) THEN "BAJA+1"
WHEN t1.foto_mes < 202105 AND t1.foto_mes+2 IN 
    (SELECT t2.foto_mes
    FROM df t2
    WHERE t2.numero_de_cliente = t1.numero_de_cliente) THEN "CONTINUA"
ELSE NULL END AS Target
FROM df


FROM df t1

In [19]:
%%sql
select numero_de_cliente,
CASE WHEN cliente_vip = 1 THEN 'cliente vip' ELSE 'cliente pobre' END AS cliente_cheto
from df 


,numero_de_cliente,cliente_cheto
0,249221323,cliente pobre
1,249227600,cliente pobre
2,249234235,cliente pobre
3,249244449,cliente pobre
4,249244739,cliente pobre
...,...,...
978434,1598368433,cliente pobre
978435,1598419415,cliente pobre
978436,1598425905,cliente pobre
978437,1598470389,cliente pobre


In [26]:
%%sql
SELECT numero_de_cliente , mrentabilidad , mrentabilidad_annual ,mrentabilidad + mrentabilidad_annual AS sumatoria
FROM df 

,numero_de_cliente,mrentabilidad,mrentabilidad_annual,sumatoria
0,249221323,3070.24,15691.10,18761.34
1,249227600,496.21,15321.80,15818.01
2,249234235,806.56,10549.80,11356.36
3,249244449,27485.71,65621.74,93107.45
4,249244739,3306.82,48787.99,52094.81
...,...,...,...,...
978434,1598368433,4123.09,4123.09,8246.18
978435,1598419415,-2.26,-2.26,-4.52
978436,1598425905,5.92,5.92,11.84
978437,1598470389,244.96,244.96,489.92


In [ ]:
%%sql
SELECT numero_de_cliente, COUNT()
FROM df 
GROUP BY numero_de_cliente
HAVING COUNT(*) > 5

,numero_de_cliente
0,1073610732
1,1073699050
2,1073963773
3,1074111886
4,1074276489
...,...
156995,917814459
156996,917881770
156997,917960899
156998,917970523


In [3]:
%%sql
SELECT DISTINCT(foto_mes)
FROM df
ORDER BY foto_mes

,foto_mes
0,202101
1,202102
2,202103
3,202104
4,202105
5,202106


In [4]:
%%sql
SELECT t1.numero_de_cliente, t1.foto_mes, 
CASE WHEN t1.foto_mes <202106 AND  t1.foto_mes +1  NOT IN 
    (SELECT t2.foto_mes
    FROM df as t2
    WHERE t1.numero_de_cliente = t2.numero_de_cliente)  THEN 'BAJA +1'
WHEN t1.foto_mes <202105 AND t1.foto_mes + 2 NOT IN 
    (SELECT t2.foto_mes
    FROM df as t2
    WHERE t1.numero_de_cliente = t2.numero_de_cliente)  THEN 'BAJA +2'
WHEN t1.foto_mes <202105 AND t1.foto_mes + 2  IN 
    (SELECT t2.foto_mes
    FROM df as t2
    WHERE t1.numero_de_cliente = t2.numero_de_cliente)  THEN 'Continua'
ELSE NULL END AS Baja
FROM df as t1

,numero_de_cliente,foto_mes,Baja
0,1147706966,202105,None
1,1147712585,202105,None
2,1147715061,202105,None
3,1147720091,202105,None
4,1147729502,202105,None
...,...,...,...
978434,1078806494,202106,None
978435,1078815392,202106,None
978436,1078831293,202106,None
978437,1078838731,202106,None


In [ ]:
%%sql

SELECT *,
CASE WHEN t1.foto_mes < 202106 AND t1.foto_mes +1 NOT IN 
    (SELECT t2.foto_mes
    FROM df as t2
    WHERE t1.numero_de_cliente = t2.numero_de_cliente) THEN 'BAJA+1'
WHEN t1.foto_mes < 202105 AND t1.foto_mes NOT IN 
    (SELECT t2.foto_mes
    FROM df as t2 
    WHERE t1.numero_de_cliente = t2.numero_de_cliente) THEN 'BAJA+2'
WHEN t1.foto_mes < 202105 AND t1.foto_mes IN 
    (SELECT t2.foto_mes 
    FROM df as t2
    WHERE t1.numero_de_cliente = t2.numero_de_cliente) THEN 'Continua'
ELSE NULL END target
FROM df as t1

,numero_de_cliente,foto_mes,target
0,1072717672,202101,Continua
1,1072717886,202101,Continua
2,1072720516,202101,Continua
3,1072723872,202101,Continua
4,1072728244,202101,Continua
...,...,...,...
978434,918082906,202101,Continua
978435,918086117,202101,Continua
978436,918091369,202101,Continua
978437,918092018,202101,Continua


In [ ]:
%%sql

SELECT foto_mes , 
COUNT(*) FILTER (WHERE Baja = 'BAJA +1') as 'BAJA+1',
COUNT(*) FILTER (WHERE Baja='BAJA +2') as 'BAJA+2',
COUNT(*) FILTER (WHERE Baja = 'Continua') as 'Continua'
FROM
(SELECT t1.numero_de_cliente, t1.foto_mes, 
CASE WHEN t1.foto_mes < 202106 AND  t1.foto_mes +1  NOT IN 
    (SELECT t2.foto_mes
    FROM df as t2
    WHERE t1.numero_de_cliente = t2.numero_de_cliente)  THEN 'BAJA +1'
WHEN t1.foto_mes <202105 AND t1.foto_mes + 2 NOT IN 
    (SELECT t2.foto_mes
    FROM df as t2
    WHERE t1.numero_de_cliente = t2.numero_de_cliente)  THEN 'BAJA +2'
WHEN t1.foto_mes <202105 AND t1.foto_mes + 2  IN 
    (SELECT t2.foto_mes
    FROM df as t2
    WHERE t1.numero_de_cliente = t2.numero_de_cliente)  THEN 'Continua'
ELSE NULL END AS Target
FROM df as t1)
GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,Continua
0,202101,622,825,160080
1,202102,831,1032,160292
2,202103,1039,951,161119
3,202104,955,1130,161333
4,202105,1134,0,0
5,202106,0,0,0


In [ ]:
%%sql

SELECT foto_mes , 
COUNT(*) FILTER (WHERE Baja = 'BAJA +1') as 'BAJA+1',
COUNT(*) FILTER (WHERE Baja='BAJA +2') as 'BAJA+2',
COUNT(*) FILTER (WHERE Baja = 'Continua') as 'Continua'
FROM
(SELECT t1.numero_de_cliente, t1.foto_mes, 
CASE WHEN t1.foto_mes < 
    (SELECT DISTINCT(foto_mes)
    FROM df 
    ORDER BY foto_mes DESC
    LIMIT 1)
AND  t1.foto_mes +1  NOT IN 
    (SELECT t2.foto_mes
    FROM df as t2
    WHERE t1.numero_de_cliente = t2.numero_de_cliente)  THEN 'BAJA +1'
WHEN t1.foto_mes <202105 AND t1.foto_mes + 2 NOT IN 
    (SELECT t2.foto_mes
    FROM df as t2
    WHERE t1.numero_de_cliente = t2.numero_de_cliente)  THEN 'BAJA +2'
WHEN t1.foto_mes <202105 AND t1.foto_mes + 2  IN 
    (SELECT t2.foto_mes
    FROM df as t2
    WHERE t1.numero_de_cliente = t2.numero_de_cliente)  THEN 'Continua'
ELSE NULL END AS Target
FROM df as t1)
GROUP BY foto_mes

In [47]:
%%sql
SELECT *, 
CASE WHEN t1.foto_mes <202106 AND  t1.foto_mes +1  NOT IN 
    (SELECT t2.foto_mes
    FROM df as t2
    WHERE t1.numero_de_cliente = t2.numero_de_cliente)  THEN 'BAJA +1'
WHEN t1.foto_mes <202105 AND t1.foto_mes + 2 NOT IN 
    (SELECT t2.foto_mes
    FROM df as t2
    WHERE t1.numero_de_cliente = t2.numero_de_cliente)  THEN 'BAJA +2'
WHEN t1.foto_mes <202105 AND t1.foto_mes + 2  IN 
    (SELECT t2.foto_mes
    FROM df as t2
    WHERE t1.numero_de_cliente = t2.numero_de_cliente)  THEN 'Continua'
ELSE NULL END AS Baja
FROM df as t1

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,Baja
0,310287459,202104,1,0,0,62,281,12829.51,141464.22,79.85,...,2,0.00,NaN,NaN,3182,NaN,<NA>,<NA>,0.00,Continua
1,310293437,202104,1,0,0,76,323,7187.41,88445.65,786.12,...,2,0.00,NaN,NaN,2024,NaN,<NA>,<NA>,0.00,Continua
2,310296281,202104,1,0,0,42,200,3242.01,50815.73,102.67,...,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>,NaN,Continua
3,310298032,202104,1,0,0,62,297,-7096.26,-27063.65,3641.91,...,23,48788.10,-47216.49,0.0,9047,6072.04,1,0,7999.86,Continua
4,310302336,202104,1,0,0,53,315,-4293.70,-47424.37,1895.04,...,23,15656.69,0.00,0.0,7372,2602.30,1,0,0.00,Continua
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978434,1182363291,202102,1,0,0,40,6,-83.40,-691.13,112.38,...,-3,0.00,-12778.28,0.0,158,17937.71,4,0,0.00,Continua
978435,1182365041,202102,1,0,0,35,57,-424.19,10126.68,1939.35,...,-3,0.00,-27698.84,0.0,1739,11575.21,5,0,8562.90,Continua
978436,1182370584,202102,1,0,0,33,57,4230.96,35098.41,2037.64,...,-3,0.00,NaN,NaN,1713,NaN,<NA>,<NA>,0.00,Continua
978437,1182371096,202102,1,0,0,56,56,1529.82,13287.55,1362.79,...,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>,NaN,Continua


In [9]:
%%sql
COPY (
    SELECT *, 
    CASE WHEN t1.foto_mes <202106 AND  t1.foto_mes +1  NOT IN 
        (SELECT t2.foto_mes
        FROM df as t2
        WHERE t1.numero_de_cliente = t2.numero_de_cliente)  THEN 'BAJA +1'
    WHEN t1.foto_mes <202105 AND t1.foto_mes + 2 NOT IN 
        (SELECT t2.foto_mes
        FROM df as t2
        WHERE t1.numero_de_cliente = t2.numero_de_cliente)  THEN 'BAJA +2'
    WHEN t1.foto_mes <202105 AND t1.foto_mes + 2  IN 
        (SELECT t2.foto_mes
        FROM df as t2
        WHERE t1.numero_de_cliente = t2.numero_de_cliente)  THEN 'Continua'
    ELSE NULL END AS clase_ternaria
    FROM df as t1
) 
TO 'competencia_01.csv' (HEADER, DELIMITER ',')


,Count
0,978439


# 2do INTENTO

In [4]:
%%sql
SELECT foto_mes ,numero_de_cliente,
lead(foto_mes , 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_1,
lead(foto_mes , 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_2,
FROM df 


,foto_mes,numero_de_cliente,foto_mes_1,foto_mes_2
0,202101,249244449,202102,202103
1,202102,249244449,202103,202104
2,202103,249244449,202104,202105
3,202104,249244449,202105,202106
4,202105,249244449,202106,<NA>
...,...,...,...,...
978434,202105,1595472208,202106,<NA>
978435,202106,1595472208,<NA>,<NA>
978436,202106,1595983866,<NA>,<NA>
978437,202106,1596979317,<NA>,<NA>


In [9]:
%%sql
with df2 as (
    SELECT foto_mes , numero_de_cliente,
    lead(foto_mes  , 1 ) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_1,
    lead(foto_mes  , 2 ) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_2
    FROM df
)
SELECT d.numero_de_cliente,d.foto_mes , d.foto_mes_1,d.foto_mes_2
FROM df2 d
WHERE d.foto_mes_1 IS NULL AND  d.foto_mes_2 IS NOT NULL

,numero_de_cliente,foto_mes,foto_mes_1,foto_mes_2


In [10]:
%%sql
with df2 as (
    SELECT foto_mes , numero_de_cliente,
    lead(foto_mes  , 1 ) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_1,
    lead(foto_mes  , 2 ) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_2
    FROM df
)
SELECT d.numero_de_cliente,d.foto_mes , d.foto_mes_1,d.foto_mes_2
FROM df2 d
WHERE d.foto_mes =202105  AND  d.foto_mes_1 IS NULL

,numero_de_cliente,foto_mes,foto_mes_1,foto_mes_2
0,638279479,202105,<NA>,<NA>
1,643465907,202105,<NA>,<NA>
2,654555709,202105,<NA>,<NA>
3,683390557,202105,<NA>,<NA>
4,688620743,202105,<NA>,<NA>
...,...,...,...,...
1129,684934934,202105,<NA>,<NA>
1130,719933292,202105,<NA>,<NA>
1131,724191281,202105,<NA>,<NA>
1132,773802978,202105,<NA>,<NA>


In [14]:
%%sql
with df2 as (
    SELECT foto_mes , numero_de_cliente,
    lead(foto_mes  , 1 ) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_1,
    lead(foto_mes  , 2 ) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_2
    FROM df  
)
SELECT *,
if (foto_mes < 202106 , if(foto_mes <202105 ,if(df2.foto_mes_1 IS NULL,'BAJA+1', if(df2.foto_mes_2 IS NULL,'BAJA+2','CONTINUA')) ,if(df2.foto_mes_1 IS NULL,'BAJA+1',NULL)) ,NULL) as target
from df
LEFT JOIN df2 USING (numero_de_cliente,foto_mes)

 

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,foto_mes_1,foto_mes_2,target
0,304861176,202103,1,0,0,44,291,5749.90,81043.49,1825.89,...,-37130.14,39.30,7672,24661.79,8,0,1442.79,202104,202105,CONTINUA
1,304862491,202103,1,0,0,70,375,-1400.85,2156.67,3518.54,...,-89275.91,2.33,9219,49520.24,24,0,6111.33,202104,202105,CONTINUA
2,304865693,202103,1,0,0,80,286,3080.50,29686.10,2839.10,...,NaN,NaN,4551,NaN,<NA>,<NA>,0.00,202104,202105,CONTINUA
3,304874882,202103,1,0,0,74,204,5607.23,57499.08,431.98,...,-27204.56,0.00,6184,26881.98,2,0,1395.87,202104,202105,CONTINUA
4,304877879,202103,1,0,0,52,313,1210.05,5344.52,1754.69,...,-4287.91,0.00,8309,4907.23,3,0,469.20,202104,202105,CONTINUA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978434,1598368433,202106,1,0,0,58,1,4123.09,4123.09,2242.51,...,0.00,0.00,28,12521.76,2,0,1747.77,<NA>,<NA>,None
978435,1598419415,202106,1,0,0,30,1,-2.26,-2.26,0.00,...,NaN,NaN,9,NaN,<NA>,<NA>,0.00,<NA>,<NA>,None
978436,1598425905,202106,1,0,0,28,1,5.92,5.92,0.00,...,NaN,NaN,20,NaN,<NA>,<NA>,0.00,<NA>,<NA>,None
978437,1598470389,202106,1,0,0,78,1,244.96,244.96,0.00,...,NaN,NaN,15,NaN,<NA>,<NA>,0.00,<NA>,<NA>,None


In [ ]:
%%sql
SELECT foto_mes , 
COUNT(*) FILTER (WHERE target = 'BAJA+1') as 'BAJA+1',
COUNT(*) FILTER (WHERE target='BAJA+2') as 'BAJA+2',
COUNT(*) FILTER (WHERE target = 'Continua') as 'Continua'
FROM
(
with df2 as (
    SELECT foto_mes , numero_de_cliente,
    lead(foto_mes  , 1 ) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_1,
    lead(foto_mes  , 2 ) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_2
    FROM df  
)
SELECT *,
if (foto_mes < 202106 , if(foto_mes <202105 ,
if(df2.foto_mes_1 IS NULL,'BAJA+1', 
if(df2.foto_mes_2 IS NULL,'BAJA+2','Continua')) ,
if(df2.foto_mes_1 IS NULL,'BAJA+1',NULL)) ,NULL) as target
from df
LEFT JOIN df2 USING (numero_de_cliente,foto_mes)

 )
 GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,Continua
0,202101,616,824,160087
1,202102,830,1027,160298
2,202103,1034,953,161122
3,202104,954,1131,161333
4,202105,1134,0,0
5,202106,0,0,0


In [20]:
%%sql
with df2 as (
    SELECT foto_mes , numero_de_cliente,
    lead(foto_mes  , 1 ) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_1,
    lead(foto_mes  , 2 ) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_2
    FROM df  
)
SELECT * EXCLUDE (foto_mes_1,foto_mes_2),
if (foto_mes < 202106 , if(foto_mes <202105 ,
if(df2.foto_mes_1 IS NULL,'BAJA+1', 
if(df2.foto_mes_2 IS NULL,'BAJA+2','Continua')) ,
if(df2.foto_mes_1 IS NULL,'BAJA+1',NULL)) ,NULL) as target
from df
LEFT JOIN df2 USING (numero_de_cliente,foto_mes)

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,target
0,249221323,202101,1,0,0,46,93,3070.24,15691.10,1278.11,...,4,0.00,-20833.47,0.0,2814,6743.38,6,0,9559.95,Continua
1,249227600,202101,1,0,0,42,278,496.21,15321.80,418.97,...,11,13267.49,-9662.51,0.0,5541,9419.90,8,0,1900.26,Continua
2,249234235,202101,1,0,0,54,17,806.56,10549.80,174.26,...,4,0.00,-4692.00,0.0,577,2821.32,2,0,6885.51,Continua
3,249244449,202101,1,0,0,61,290,27485.71,65621.74,23590.68,...,25,23800.17,-20349.22,0.0,7916,23509.71,8,0,1219.92,Continua
4,249244739,202101,1,0,0,66,320,3306.82,48787.99,3274.95,...,4,0.00,-88178.37,0.0,1958,20397.54,8,0,2627.52,Continua
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978434,918082906,202101,1,0,0,32,120,7915.97,30422.44,2009.59,...,4,0.00,-8740.02,0.0,3647,7792.18,4,0,1161.27,Continua
978435,918086117,202101,1,0,0,61,120,-1054.04,-6722.55,569.89,...,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>,NaN,Continua
978436,918091369,202101,1,0,0,38,120,244.13,2352.58,623.75,...,4,0.00,-7284.52,0.0,3154,709.82,1,0,1736.04,Continua
978437,918092018,202101,1,0,0,50,119,11.07,-3424.73,1922.37,...,25,13724.10,-44738.22,0.0,3617,26328.70,26,0,6463.23,BAJA+2


In [21]:
%%sql
SELECT foto_mes , 
COUNT(*) FILTER (WHERE target = 'BAJA+1') as 'BAJA+1',
COUNT(*) FILTER (WHERE target='BAJA+2') as 'BAJA+2',
COUNT(*) FILTER (WHERE target = 'Continua') as 'Continua'
FROM
(
with df2 as (
    SELECT foto_mes , numero_de_cliente,
    lead(foto_mes  , 1 ) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_1,
    lead(foto_mes  , 2 ) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_2
    FROM df  
)
SELECT * EXCLUDE (foto_mes_1,foto_mes_2),
if (foto_mes < 202106 , if(foto_mes <202105 ,
if(df2.foto_mes_1 IS NULL,'BAJA+1', 
if(df2.foto_mes_2 IS NULL,'BAJA+2','Continua')) ,
if(df2.foto_mes_1 IS NULL,'BAJA+1',NULL)) ,NULL) as target
from df
LEFT JOIN df2 USING (numero_de_cliente,foto_mes)

 )
 GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,Continua
0,202101,616,824,160087
1,202102,830,1027,160298
2,202103,1034,953,161122
3,202104,954,1131,161333
4,202105,1134,0,0
5,202106,0,0,0


In [9]:
%%sql
COPY (with df2 as (
    SELECT foto_mes , numero_de_cliente,
    lead(foto_mes  , 1 ) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_1,
    lead(foto_mes  , 2 ) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as foto_mes_2
    FROM df  
)
SELECT * EXCLUDE (foto_mes_1,foto_mes_2),
if (foto_mes < 202106 , if(foto_mes <202105 ,
if(df2.foto_mes_1 IS NULL,'BAJA+1', 
if(df2.foto_mes_2 IS NULL,'BAJA+2','Continua')) ,
if(df2.foto_mes_1 IS NULL,'BAJA+1',NULL)) ,NULL) as clase_ternaria
from df
LEFT JOIN df2 USING (numero_de_cliente,foto_mes)) 
TO '../data/competencia_01.csv' (HEADER, DELIMITER ',')

,Count
0,978439


In [12]:
%%sql

select numero_de_cliente , foto_mes,
lead(foto_mes,1) over (partition by numero_de_cliente order by foto_mes) as foto_mes_1,
lead(foto_mes,2) over (partition by numero_de_cliente order by foto_mes) as foto_mes_2
from df

,numero_de_cliente,foto_mes,foto_mes_1,foto_mes_2
0,249274782,202101,202102,202103
1,249274782,202102,202103,202104
2,249274782,202103,202104,202105
3,249274782,202104,202105,202106
4,249274782,202105,202106,<NA>
...,...,...,...,...
978434,844971047,202105,202106,<NA>
978435,844971047,202106,<NA>,<NA>
978436,845074532,202101,202102,202103
978437,845074532,202102,202103,202104


In [13]:
from src.creacion_target import creacion_clase_ternaria
import polars as pl

df=pl.read_csv("../datasets/competencia_01_crudo.csv")

df = creacion_clase_ternaria(df  , 2)



ModuleNotFoundError: No module named 'src'